In [20]:
# %pip install plotly
# %pip install nbformat

In [21]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_dmon(filename, GPUIds=None, Fields=None):
    # Load the CSV data
    df = pd.read_csv(filename)
    
    # Convert TimeStampUs to a human-readable datetime format
    df['TimeStampUs'] = pd.to_datetime(df['TimeStampUs'], unit='us')
    
    # Apply filters if provided
    if GPUIds is not None:
        df = df[df['GPUId'].isin(GPUIds)]
    
    # Determine the number of rows and columns for the subplots
    field_groups=[['.*']]
    rows=len(field_groups)
    if Fields:
        field_groups=Fields
        rows = len(Fields)
    
    # Create a Plotly figure with subplots
    fig = make_subplots(rows=rows, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.1, horizontal_spacing=0.05,
                        subplot_titles=[item for sublist in field_groups for item in sublist])
    
    # Add traces for each GPUId and each Field in the subplots
    for i, field_group in enumerate(field_groups):
        #for j, field in enumerate(field_group):
        for field_regex in field_group:
            field_subset = df[df['Field'].str.contains(field_regex, regex=True)]
            if field_subset.empty:
                continue
            for gpu_id in field_subset['GPUId'].unique():
                for field in field_subset['Field'].unique():
                    subset = field_subset[(field_subset['GPUId'] == gpu_id) & (field_subset['Field'] == field)]
                    if subset.empty:
                        continue
                    fig.add_trace(go.Scatter(
                        x=subset['TimeStampUs'],
                        y=subset['Values'],
                        mode= 'markers' if field.startswith('throttle_reason') else 'lines+markers',
                        name=f'GPU {gpu_id} - {field}'
                    ),  row=i+1, col=1)
    
    # Update layout
    fig.update_layout(
        title='GPU Data Plot',
        xaxis_title='Timestamp',
        yaxis_title='Values',
        legend_title='GPUId and Field',
        height=300*rows  # Adjust height to ensure enough space for subplots
    )
    
    # Show the plot
    fig.show()

# Example usage
# dmon_plot('path_to_csv_file.csv', GPUIds=[0, 1], Fields=[['power_usage_instant'], ['current_clock_throttle_reasons', 'other_field']])



In [22]:
# The diagram is not really readable due to variable ranges for different metrics.
# But it provides the list of metrics
plot_dmon("dmon.csv")

In [24]:
# Pick the data from GPU0
# And divide them into 2 different subplots
GPUIds=[0]
fields=[
    ['power_usage_instant'],
    ['current_clock_throttle_reasons'],
    ['^throttle_reason_*'] # break current_clock_throttle_reasons into different metrics
]
plot_dmon("dmon.csv", GPUIds=GPUIds, Fields=fields)